In [2]:
import pandas as pd
import numpy as np
from datetime import date
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator, ZeroCount
import math
import os  
mingw_path = 'D:\MINGW\mingw64\bin'  
os.environ['PATH'] = mingw_path + ';' + os.environ['PATH'] 
import xgboost as xgb 
from xgboost import XGBRegressor




def deletezero(x):
    if x<3.8:
        result = 0
    else:
        result = x
    return result
 
read_x1 = pd.read_csv('D:/研究生/Chaos/JD/信贷需求/traindata/traindata.csv')
read_y1 = pd.read_csv('D:/研究生/Chaos/JD/信贷需求/traindata/traindata_label.csv')
read_x2 = pd.read_csv('D:/研究生/Chaos/JD/信贷需求/testdata/testdata.csv')







d:\python3.5\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
# x1 = read_x1[['loan_8_sum','loan_9_sum','loan_10_sum','loan_8_mean','loan_9_mean','loan_10_mean',\
#               'loan_price_sum','topay_thismonth','topay_thismonth1','repayment_ability','last_loan_amount',\
#               'click_count_sum','order_count_sum','click_model']] 
x1 = read_x1
y1 = read_y1
y1['loan_11_sum'] = y1.loan_11_sum.apply(lambda x:1 if x>0 else 0)
train_x, valid_x, train_y, valid_y = train_test_split(x1,y1, test_size=0.0001,random_state=0)  



dataset1 = xgb.DMatrix(train_x,label=train_y)
dataset_valid = xgb.DMatrix(valid_x,label=valid_y)
param = {  
         'eta':0.1,
         'subsample':0.7,
         'objective':'binary:logistic' }    
evallist  = [(dataset_valid,'eval'), (dataset1,'train')]   
#evallist  = [(dataset1,'train')] 
num_round = 30
bst = xgb.train(param, dataset1, num_round,evallist ) 


# x2 = read_x2[['uid','loan_8_sum','loan_9_sum','loan_10_sum','loan_8_mean','loan_9_mean','loan_10_mean',\
#               'loan_price_sum','topay_thismonth','topay_thismonth1','repayment_ability','last_loan_amount',\
#               'click_count_sum','order_count_sum','click_model']] 
x2 = read_x2
dataset3_preds = x2[['uid']]
x2.drop('uid',axis=1,inplace=True)
dataset3 = xgb.DMatrix(x2)
dataset3_preds['class'] = bst.predict(dataset3)
dataset3_preds.to_csv("my_xgb_class.csv",index=None)


[0]	eval-error:0.2	train-error:0.141092
[1]	eval-error:0.2	train-error:0.13963
[2]	eval-error:0.2	train-error:0.140356
[3]	eval-error:0.2	train-error:0.139707
[4]	eval-error:0.2	train-error:0.13907
[5]	eval-error:0.2	train-error:0.138575
[6]	eval-error:0.2	train-error:0.138334
[7]	eval-error:0.2	train-error:0.137894
[8]	eval-error:0.2	train-error:0.13741
[9]	eval-error:0.2	train-error:0.137289
[10]	eval-error:0.2	train-error:0.137399
[11]	eval-error:0.2	train-error:0.137234
[12]	eval-error:0.2	train-error:0.137136
[13]	eval-error:0.2	train-error:0.136905
[14]	eval-error:0.2	train-error:0.136784
[15]	eval-error:0.2	train-error:0.136553
[16]	eval-error:0.2	train-error:0.136542
[17]	eval-error:0.2	train-error:0.136344
[18]	eval-error:0.2	train-error:0.136278
[19]	eval-error:0.2	train-error:0.136289
[20]	eval-error:0.2	train-error:0.1363
[21]	eval-error:0.2	train-error:0.136168
[22]	eval-error:0.2	train-error:0.136146
[23]	eval-error:0.2	train-error:0.136003
[24]	eval-error:0.2	train-error

d:\python3.5\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [8]:
#save feature score
feature_score = bst.get_fscore()
feature_score = sorted(feature_score.items(), key=lambda x:x[1],reverse=True)
fs = []
for (key,value) in feature_score:
    fs.append("{0},{1}\n".format(key,value))
    
with open('xgb_classify_feature_score.csv','w') as f:
    f.writelines("feature,score\n")
    f.writelines(fs)

In [7]:
t = valid_y[valid_y.loan_11_sum<0]
print(t)

Empty DataFrame
Columns: [loan_11_sum]
Index: []
